# 02-03 SQL-Musterlösung

## Hinweise zur Übung

Ziel dieser Aufgabe ist, konkrete Abfragen gegen das Kurs-DWH zu stellen und dabei schrittweise von einfachen zu komplexeren Abfragen Erfahrung zu sammeln.

## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet ipython-sql

In [ ]:
import os
import sys
import urllib.request
import gzip
import shutil
%load_ext sql

In [ ]:
# Konfiguration
base_url_quellen   = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/08-Datenanalyse+Visualisierung/data"
base_url_reporting = "./"

In [ ]:
# SQlite-Datenbanken aus Github auf den Jupyter-Server herunterladen
urllib.request.urlretrieve(base_url_quellen + "/dwh/reporting.sqlite.gz", base_url_reporting + "reporting.sqlite.gz")

# Die Sqlite-Datenbank ist aufgrund ihrer Größe gezipped und muss vor der Nutzung noch entpackt werden
with gzip.open(base_url_reporting + "reporting.sqlite.gz", "rb") as f_in:
    with open(base_url_reporting + "reporting.sqlite", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Datenbankverbindung als Pfad (für das ETL) & iPython SQL (für die Abfragen) herstellen
db_path_reporting      = base_url_reporting + "reporting.sqlite"

db_url_reporting      = "sqlite:///" + db_path_reporting

%sql $db_url_reporting

## Einfache Abfragen über einzelne Tabellen

### Aufgabe: Fragen Sie alle Spalten der Tabelle `F_FAELLE` ab

🛑 Denken Sie daran, die Abfrage mit einer `LIMIT`-Klausel auf wenige Zeilen zu beschränken, da sonst alle Datensätze geladen würden (lange Laufzeit, ggf. Absturz des Notebooks). :

In [ ]:
# Alle Spalten der Tabelle F_FAELLE ausgeben (beschränkt auf 10 Zeilen)
%%sql $db_url_reporting
SELECT *
  FROM f_faelle
 LIMIT 10

### Aufgabe: Fragen Sie nur die Spalten `fall_id`, `aufnahme_datum` und `entlass_datum` der Tabelle `F_FAELLE` ab

In [ ]:
# Spalten fall_id, aufnahme_datum und entlass_datum der Tabelle F_FAELLE ausgeben (beschränkt auf 10 Zeilen)
%%sql $db_url_reporting
SELECT fall_id,
       aufnahme_datum,
       entlass_datum
  FROM f_faelle
 LIMIT 10

### Aufgabe: Fragen Sie männliche Patienten aus der Tabelle `D_PATIENT` ab (alle Spalten)

In [ ]:
# Männliche Patienten aus der Tabelle D_PATIENT abfragen (beschränkt auf 10 Zeilen)
%%sql $db_url_reporting
SELECT *
  FROM d_patient
 WHERE patient_geschlecht = 'M'
 LIMIT 10

## Abfragen über mehrere verbundene Tabellen

### Aufgabe: Joinen Sie die Tabellen D_PATIENT und F_FAELLE (INNER JOIN) und fragen alle Spalten ab

💡 Denken Sie daran, für beide Tabellen jeweils ein kurzes Alias zu definieren und das Alias zur Qualifizierung der Spalten beider Tabellen zu nutzen.

In [ ]:
# Inner Join der Tabellen D_PATIENT und F_FAELLE (beschränkt auf 10 Zeilen)
%%sql $db_url_reporting
SELECT *
  FROM d_patient pat
  JOIN f_faelle  fal ON pat.patient_id = fal.patient_id
 LIMIT 10

### Aufgabe: Ergänzen Sie die Abfrage um einen INNER JOIN mit der Tabelle D_FALLART

Geben Sie nur die folgenden Spalten aus:
* D_PATIENT: patient_id, patient_nachname
* F_FAELLE: aufnahme_datum, entlass_datum
* D_FALLART: fallart_name

In [ ]:
# Inner Join der Tabellen D_PATIENT, F_FAELLE und D_FALLART (beschränkt auf 10 Zeilen)
%%sql $db_url_reporting
SELECT pat.patient_id,
       pat.patient_nachname,
       fal.aufnahme_datum,
       fal.entlass_datum,
       frt.fallart_name
  FROM d_patient pat
  JOIN f_faelle  fal ON pat.patient_id = fal.patient_id
  JOIN d_fallart frt ON fal.fallart_id = frt.fallart_id
 LIMIT 10

### Aufgabe: Ergänzen Sie die vorherige Abfrage um einen LEFT OUTER JOIN, um den Bezeichner der Hauptdiagnose anzuzeigen

Achten Sie darauf, dass Fälle mit fehlender Hauptdiagnose weiterhin sichtbar bleiben.

In [ ]:
# Inner Join der Tabellen D_PATIENT, F_FAELLE und D_FALLART mit LEFT JOIN zur Tabelle D_DIAGNOSE (beschränkt auf 10 Zeilen)
%%sql $db_url_reporting
SELECT pat.patient_id,
       pat.patient_nachname,
       fal.aufnahme_datum,
       fal.entlass_datum,
       frt.fallart_name,
       dkt.snomed_name
  FROM      d_patient  pat
  JOIN      f_faelle   fal ON pat.patient_id              = fal.patient_id
  JOIN      d_fallart  frt ON fal.fallart_id              = frt.fallart_id
  LEFT JOIN d_diagnose dkt ON fal.hauptdiagnose_snomed_id = dkt.snomed_id
 LIMIT 10

### Aufgabe: Geben Sie Einrichtungen aus, die keine Behandlungsfälle hatten

In [ ]:
# LEFT JOIN der Fälle an die Einrichtungen und Filterung auf Einträge ohne zugehörige Fälle
%%sql $db_url_reporting
SELECT *
  FROM d_orga org
  LEFT JOIN f_faelle fal ON org.einrichtung_id = fal.einrichtung_id
 WHERE fal.einrichtung_id IS NULL
 LIMIT 10

## Aggregation von Daten

In den folgenden Abfragen benötigen Sie Gruppierungsmerkmale und Aggregatfunktionen, um Daten über mehrere Tabellenzeilen hinweg zusammengefasst auszuwerten.

### Aufgabe: Berechnen Sie den aufsummierten Gesamterlös pro Einrichtung

Geben Sie dazu die die folgenden Spalten aus:
* D_ORGA: `standort_name`,  `einrichtung_name`
* F_FAELLE: Summe von `erloes_fallpauschale`

In [ ]:
# Abfrage der Tabellen D_ORGA und F_FAELLE mit Aggregation des Erlöses
%%sql $db_url_reporting
SELECT org.standort_name,
       org.einrichtung_name,
       SUM(fal.erloes_fallpauschale) AS erloes_sum
  FROM d_orga   org
  JOIN f_faelle fal ON org.einrichtung_id = fal.einrichtung_id
 GROUP BY org.standort_name,
       org.einrichtung_name
 LIMIT 10

### Aufgabe: Ergänzen Sie die Abfrage um die Fallzahl und den mittleren Erlös und filtern auf stationäre Fälle

Stationäre Fälle haben die Fallart "inpatient"

In [ ]:
# Abfrage der Tabellen D_ORGA und F_FAELLE mit Aggregation des Erlöses
%%sql $db_url_reporting
SELECT org.standort_name,
       org.einrichtung_name,
       COUNT(*)                      AS fallzahl,
       SUM(fal.erloes_fallpauschale) AS erloes_sum,
       AVG(fal.erloes_fallpauschale) AS erloes_avg
  FROM d_orga    org
  JOIN f_faelle  fal ON org.einrichtung_id = fal.einrichtung_id
  JOIN d_fallart frt ON fal.fallart_id     = frt.fallart_id
 WHERE frt.fallart_name = 'inpatient'
 GROUP BY org.standort_name,
       org.einrichtung_name
 LIMIT 10

### Aufgabe: Ergänzen Sie die vorherige Abfrage um einen Filter auf Einrichtungen mit mehr als 50 Fällen und sortieren das Ergebnis absteigend nach Gesamterlös

In [ ]:
# Abfrage der Tabellen D_ORGA und F_FAELLE mit Aggregation des Erlöses
%%sql $db_url_reporting
SELECT org.standort_name,
       org.einrichtung_name,
       COUNT(*)                      AS fallzahl,
       SUM(fal.erloes_fallpauschale) AS erloes_sum,
       AVG(fal.erloes_fallpauschale) AS erloes_avg
  FROM d_orga    org
  JOIN f_faelle  fal ON org.einrichtung_id = fal.einrichtung_id
  JOIN d_fallart frt ON fal.fallart_id     = frt.fallart_id
 WHERE frt.fallart_name = 'inpatient'
 GROUP BY org.standort_name,
       org.einrichtung_name
HAVING COUNT(*) > 50
 ORDER BY SUM(fal.erloes_fallpauschale) DESC
 LIMIT 100